In [1]:
import numpy as np
import itertools
import csv

In [20]:
# Experiment Design
conditions = ['std', 'tar', 'odd']
n_sbj = 30
n_trials = 130
n_std = 98
n_tar = 16
n_odd = 16
n_blocks = 3       # 2 for ECoG, 3 for EEG

# Response Randomization

In [3]:
# # Randomizing stimulus and response assignments
# resp_opts = [0, 1]
# cond_opts = [0, 1, 2]
# resp_perms = list(itertools.permutations(resp_opts))
# cond_perms = list(itertools.permutations(cond_opts))

# np.random.permutation(np.arange(len(conditions)))

array([2, 0, 1])

### I'm not using this, just going in order and cycling through the permutations to make sure they're evenly covered

# Sequence Randomization

## Real Task

In [31]:
# Randomization Constraints
first_possible = 3    # (python index of) first trial that can be surprise
min_gap = 3    # minimum trials between surprising outcomes

assert first_possible < n_trials-(n_odd*min_gap)

In [43]:
b2b_tests = [False] * len(tar_ix)
for t_ix in range(len(tar_ix)):
    if any(np.abs(odd_ix-tar_ix[t_ix])<2):
        b2b_tests[t_ix] = True
print odd_ix
print tar_ix
print np.abs(odd_ix-tar_ix[t_ix])
print b2b_tests

[  3   4   7  11  35  38  41  52  79  85  91  93  96 101 107 117]
[  2  23  30  32  43  45  53  59  62  63  65  73  81  97 104 106]
[103 102  99  95  71  68  65  54  27  21  15  13  10   5   1  11]
[True, False, False, False, False, False, True, False, False, False, False, False, False, True, False, True]


In [44]:
# Create list of trial types to randomize
cond_n = np.concatenate((np.tile(0,n_std),np.tile(1,n_tar),np.tile(2,n_odd))).astype(int)

# Create randomized trial orders
cond_blocks = np.zeros([n_blocks,n_trials])
rand_iter = 1
for b in range(n_blocks):
    cond_blocks[b,:] = np.random.permutation(cond_n)
    # Criteria:
    #   not starting on target or oddball
    #   no targets or oddballs within min_gap of last
    #   no targets or oddballs back-to-back
    b2b = False
    tar_ix = np.where(cond_blocks[b,:]==1)[0]
    odd_ix = np.where(cond_blocks[b,:]==2)[0]
    for t_ix in range(len(tar_ix)):
        if any(np.abs(odd_ix-tar_ix[t_ix])<2):
            b2b = True
    while np.where(cond_blocks[b,:]==1)[0][0] < first_possible or \
            np.where(cond_blocks[b,:]==2)[0][0] < first_possible or \
            (np.diff(np.where(cond_blocks[b,:]==1)) < min_gap).any() or \
            (np.diff(np.where(cond_blocks[b,:]==2)) < min_gap).any() or \
            b2b:
        cond_blocks[b,:] = np.random.permutation(cond_n)
        b2b = False
        tar_ix = np.where(cond_blocks[b,:]==1)[0]
        odd_ix = np.where(cond_blocks[b,:]==2)[0]
        for t_ix in range(len(tar_ix)):
            if any(np.abs(odd_ix-tar_ix[t_ix])<2):
                b2b = True
        rand_iter += 1
    print 'block {0} done!'.format(b)

print '# times re-randomizing = ', rand_iter

block 0 done!
block 1 done!
block 2 done!
# times re-randomizing =  48488187
0 0


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [49]:
# Check that sequences are different
for b1 in range(n_blocks):
    for b2 in range(n_blocks-b1-1):
        if all(cond_blocks[b1,:]==cond_blocks[b2+b1+1,:]):
            print 'Warning!!! blocks {0} and {1} are the same!'.format(b1,b2+b1+1)
        else:
            print 'blocks {0} and {1} are different.'.format(b1,b2+b1+1)

blocks 0 and 1 are different.
blocks 0 and 2 are different.
blocks 1 and 2 are different.


In [53]:
for b in range(n_blocks):
    print 'Block {0}:'.format(b)
    print 'first (tar, odd):', np.where(cond_blocks[b,:]==1)[0][0], np.where(cond_blocks[b,:]==2)[0][0]
    print 'min gaps (tar, odd):', np.min(np.diff(np.where(cond_blocks[b,:]==1))), np.min(np.diff(np.where(cond_blocks[b,:]==2)))
    print 'mean gaps (tar, odd):', np.mean(np.diff(np.where(cond_blocks[b,:]==1))), np.mean(np.diff(np.where(cond_blocks[b,:]==2)))
    print

Block 0:
first (tar, odd): 9 5
min gaps (tar, odd): 3 4
mean gaps (tar, odd): 7.6 7.6

Block 1:
first (tar, odd): 6 10
min gaps (tar, odd): 3 3
mean gaps (tar, odd): 8.0 7.4

Block 2:
first (tar, odd): 13 7
min gaps (tar, odd): 3 3
mean gaps (tar, odd): 7.6 8.13333333333



In [54]:
# Write output
with open('oddball_trial_order_csvs/block_' + str(n_trials) + '_trial_randomized_orders.csv', 'w') as write:
    writer = csv.writer(write)
    for b in range(n_blocks):
        writer.writerow(cond_blocks[b,:])

## Debug Blocks

In [8]:
# Create list of trial types to randomize
n_trials = 10
n_blocks = 8
n_std = 8
n_tar = 1
n_odd = 1
cond_n = np.concatenate((np.tile(0,n_std),np.tile(1,n_tar),np.tile(2,n_odd))).astype(int)

# Create randomized trial orders
cond_blocks = np.zeros([n_blocks,n_trials])
rand_iter = 1
for b in range(n_blocks):
    cond_blocks[b,:] = np.random.permutation(cond_n)
    # Criteria:  not starting on target or oddball, no targets or oddballs within min_gap of last
    while np.where(cond_blocks[b,:]==1)[0][0] < first_possible or \
            np.where(cond_blocks[b,:]==2)[0][0] < first_possible or \
            (np.diff(np.where(cond_blocks[b,:]==1)) < min_gap).any() or \
            (np.diff(np.where(cond_blocks[b,:]==2)) < min_gap).any():
        cond_blocks[b,:] = np.random.permutation(cond_n)
        rand_iter += 1
print '# times re-randomizing = ', rand_iter

# times re-randomizing =  6


In [9]:
# Write output
with open('oddball_trial_order_csvs/debug_' + str(n_trials) + '_trial_randomized_orders.csv', 'w') as write:
    writer = csv.writer(write)
    for b in range(n_blocks):
        writer.writerow(cond_blocks[b,:])